In [208]:
import pandas as pd
import numpy as np
import matplotlib


address1 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\Union.csv'
occupation_table = pd.read_csv(address1, header=0, sep = ';')
occupation_table.drop(occupation_table.columns[len(occupation_table.columns)-1], axis=1, inplace=True)

occupation_table.insert(column = 'Order', loc = 0, value = occupation_table.index)

occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005
...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W


In [209]:
address2 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\StationsLineList.csv'
stationslist = pd.read_csv(address2, header=0)
stationlist_origins = stationslist.copy()
stationlist_origins.columns = ['Line','Origins']

stationlist_destinations = stationslist.copy()
stationlist_destinations.columns = ['Line','Destinations']

stationlist_origins

,Line,Origins
0,Stouffville,Kennedy
1,Stouffville,Agincourt
2,Stouffville,Miliken
3,Stouffville,Unionville
4,Stouffville,Centennial
...,...,...
59,Lakeshore West,Aldershot
60,Lakeshore West,Hamilton
61,Lakeshore West,West Harbour
62,Lakeshore West,St. Catherines


In [189]:
address3 = r'C:\Users\lucme\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Track Occupation Data\USCR_Routes.csv'
routeslist = pd.read_csv(address3, header=0, delimiter='\t', encoding='utf16')

lineslist = routeslist.Line.str.split(pat="/",expand=True)

routeslist=pd.concat([routeslist, lineslist], axis=1, sort=False)
routeslist = routeslist.drop(['At_Node','Sequence','Line','Train Types'],axis=1).drop_duplicates(subset=['Route'])
keepcols = routeslist.columns[:4].copy().tolist()+['Line']

zero = routeslist.copy().drop([1,2],axis=1)
one = routeslist.copy().drop([0,2],axis=1)
two = routeslist.copy().drop([0,1],axis=1)

zero.columns = one.columns = two.columns = keepcols
routeslist = pd.concat([zero,one,two],axis=0)
inbound_routes = routeslist[routeslist.Direction == 'Inbound']
outbound_routes = routeslist[routeslist.Direction == 'Outbound']

inbound_routes=inbound_routes.groupby(['Line','Target'])['Route'].apply(list).to_frame().reset_index()

outbound_routes=outbound_routes.groupby(['Line','Source'])['Route'].apply(list).to_frame().reset_index()
inbound_routes.columns = ['Inbound Line','Track','Inbound Routes']
outbound_routes.columns = ['Outbound Line','Track','Outbound Routes']

inbound_routes



,Inbound Line,Track,Inbound Routes
0,Barrie,1,[BARKITCH_IN_1]
1,Barrie,12,[BARKITCH_IN_8]
2,Barrie,14,[BARKITCH_IN_9]
3,Barrie,2,[BARKITCH_IN_2]
4,Barrie,4,[BARKITCH_IN_3]
...,...,...,...
80,VIA,10,[LAKE_W_VIANYC_IN_3]
81,VIA,3,[RICHILL_IN_2]
82,VIA,5,[RICHILL_IN_3]
83,VIA,8,"[LAKE_W_VIANYC_IN_1, RICHILL_IN_4]"


In [210]:
#inbound line
occupation_table = pd.merge(occupation_table, stationlist_origins, how = 'left', on = 'Origins')
#outbound line
occupation_table = pd.merge(occupation_table, stationlist_destinations, how = 'left', on = 'Destinations')
occupation_table


,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Line_x,Line_y
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,Up Express
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,Up Express
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,Up Express
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,Up Express
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,Up Express,Up Express
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West


In [211]:
columns = occupation_table.columns.tolist()
columns[1] = 'Station'
columns[-2] = 'Inbound Line'
columns[-1] = 'Outbound Line'
occupation_table.columns = columns
occupation_table

,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,Up Express
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,Up Express
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,Up Express
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,Up Express
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,Up Express,Up Express
...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West


In [212]:
occupation_table = pd.merge( occupation_table,inbound_routes,how = 'left', on=['Inbound Line','Track'])
occupation_table = pd.merge( occupation_table,outbound_routes,how = 'left', on=['Outbound Line','Track'])
occupation_table.fillna('None')



,Order,Station,Trains,Origins,Arrival times,Track,Departure times,Destinations,Corresponding trains,Inbound Line,Outbound Line,Inbound Routes,Outbound Routes
0,0,Union Station,E4201,Mimico,04:46:00,UPE,04:55:00,UP Airport,4201,Lakeshore West,Up Express,[LAKE_W_IN_1],[UPE_OUT_2]
1,1,Union Station,E4203,Mimico,05:01:00,UPE,05:10:00,UP Airport,4203,Lakeshore West,Up Express,[LAKE_W_IN_1],[UPE_OUT_2]
2,2,Union Station,E4001,Mimico,05:14:00,UPE,05:30:00,UP Airport,4001,Lakeshore West,Up Express,[LAKE_W_IN_1],[UPE_OUT_2]
3,3,Union Station,E4003,Mimico,05:30:00,UPE,05:45:00,UP Airport,4003,Lakeshore West,Up Express,[LAKE_W_IN_1],[UPE_OUT_2]
4,4,Union Station,4002,UP Airport,05:52:00,UPE,06:00:00,UP Airport,4005,Up Express,Up Express,[UPEMILTKITCH_IN_1],[UPE_OUT_2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,381,Union Station,1224,Aldershot,17:35:00,14,17:40:00,Oshawa,9224,Lakeshore West,Lakeshore East,[LAKE_W_IN_10],"[LAKE_E_EXPSTOUFREG_OUT_11, LAKE_E_EXPSTOUFREG..."
382,382,Union Station,6232,Aurora,21:33:00,14,22:15:00,Mount Joy,7436,Barrie,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
383,383,Union Station,E7738,Mount Pleasant,22:32:00,14,23:15:00,Lincolnville,7738,Kitchener,Stouffville,[BARKITCH_IN_9],"[LAKE_E_STOUFDANF_OUT_10, LAKE_E_STOUFDANF_OUT..."
384,384,Union Station,7537,Mount Joy,23:27:00,14,23:30:00,Mimico,E7537W,Stouffville,Lakeshore West,[LAKE_E_STOUFDANF_IN_7],None


In [213]:
inRoutes = occupation_table['Inbound Routes'].apply(pd.Series)
outRoutes = occupation_table['Outbound Routes'].apply(pd.Series)


In [206]:
inRoutes

,0
0,LAKE_W_IN_1
1,LAKE_W_IN_1
2,LAKE_W_IN_1
3,LAKE_W_IN_1
4,UPEMILTKITCH_IN_1
...,...
381,LAKE_W_IN_10
382,BARKITCH_IN_9
383,BARKITCH_IN_9
384,LAKE_E_STOUFDANF_IN_7


In [214]:
occupation_table = occupation_table.drop(columns = ['Inbound Routes','Outbound Routes'], axis = 1)

In [215]:

occupation_table_out = pd.merge(occupation_table , outRoutes, left_index=True, right_index=True)
occupation_table_out = occupation_table_out.melt(id_vars = occupation_table.columns, value_name='Outbound Route').drop(columns='variable',axis=1).drop_duplicates().sort_values(by='Order')

In [216]:
occupation_table_in = pd.merge(occupation_table , inRoutes, left_index=True, right_index=True)
occupation_table_in = occupation_table_in.melt(id_vars = occupation_table.columns, value_name='Inbound Route').drop(columns='variable',axis=1).drop_duplicates().sort_values(by='Order')

In [197]:
#occupation_table_in.to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_inbound_routes_V2.csv', sep='\t',                                    encoding='UTF-16', index = False)
#occupation_table_out.to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_outbound_routes_V2.csv', sep='\t',                                  encoding='UTF-16', index = False)


In [263]:

occupation_table_out_df_list = []
for order, group in occupation_table_out.groupby(by='Order'):
    if group.shape[0] > 1:
        group.dropna(subset=['Outbound Route'], inplace=True)
        occupation_table_out_df_list.append(group)
    else:
        occupation_table_out_df_list.append(group)

occupation_table_in_df_list = []
for order, group in occupation_table_in.groupby(by='Order'):
    if group.shape[0] > 1:
        group.dropna(subset=['inbound Route'], inplace=True)
        occupation_table_in_df_list.append(group)
    else:
        occupation_table_in_df_list.append(group)
        




        
pd.concat(occupation_table_in_df_list).sort_values(by = 'Order').to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_inbound_routes_V3.csv', sep='\t',encoding='UTF-16', index = False)

pd.concat(occupation_table_out_df_list).sort_values(by = 'Order').to_csv(r'C:\Users\LUCME\Desktop\File Transfer\Desktop\Folders - Desktop\Track and Timetable Data from Treno for RER\Output\union_station_track_occupation_outbound_routes_V3.csv', sep='\t',encoding='UTF-16', index = False)